#  EDI Parser and Generator (JSON)

In [1]:
import json

In [4]:
class edi_parser:
    def __init__(self, file_path):
        self.file_path = file_path
        self.segments = []
        with open(file_path, "r") as f:
            data = f.read()
        self.segments = data.split("~")

        self.result_dict = {}
        
        self.json_data = None
        
    def parse_834(self):
        
        for segment in self.segments:
            segment_data = segment.split('*')
            key = segment_data[0]

            if key == "ISA":
                keys = self.isa_keys()
                data = self.make_dict(keys, segment_data)
            elif key == "NM1":
                keys = self.NM1_MemberName()
                data = self.make_dict(keys, segment_data)
            elif key == "PER":
                keys = self.PER_MemberCommunicationsNumbers()
                data = self.make_dict(keys, segment_data)
            elif key == "N3":
                keys = self.N3_MemberResidenceStreetAddress()
                data = self.make_dict(keys, segment_data)
            elif key == "N4":
                keys = self.N4_MemberCity_State_ZIPCode()
                data = self.make_dict(keys, segment_data)
            elif key == "DMG":
                keys = self.DMG_MemberDemographics()
                data = self.make_dict(keys, segment_data)
                
            if key in self.result_dict:
                key = f"{key}_{segment_data[1]}"

            self.result_dict[key] = data
        self.json_data = json.dumps(self.result_dict , indent = 4)
        
        print(self.json_data)
        return self.json_data
    
    def make_dict(self, keys, segment_data):
        values = [value for value in segment_data[1:]]
        return {k: v for k, v in zip(keys, values)}
    
    def isa_keys(self):
        return ["AuthorizationInformationQualifier_1",
                            "AuthorizationInformation_2",
                            "SecurityInformationQualifier_3",
                            "SecurityInformation_4",
                            "SenderIDQualifier_5",
                            "InterchangeSenderID_6",
                            "ReceiverIDQualifier_7",
                            "InterchangeReceiverID_8",
                            "InterchangeDate_9",
                            "InterchangeTime_10",
                            "InterchangeControlStandardsIdentifier_11",
                            "InterchangeControlVersionNumber_12",
                            "InterchangeControlNumber_13",
                            "AcknowledgementRequested_14",
                            "UsageIndicator_15",
                            "ComponentElementSeparator_16"]
    def NM1_MemberName(self):
        return ["EntityIdentifierCode_01",
                    "EntityTypeQualifier_02",
                    "ResponseContactLastorOrganizationName_03",
                    "ResponseContactFirstName_04",
                    "ResponseContactMiddleName_05",
                    "IdentificationCodeQualifier_08",
                    "ResponseContactIdentifier_09"]

    def PER_MemberCommunicationsNumbers(self):
        return ["ContactFunctionCode_01",
                    "CommunicationNumberQualifier_03",
                    "ResponseContactCommunicationNumber_04",
                    "CommunicationNumberQualifier_05",
                    "ResponseContactCommunicationNumber_06"]

    def N3_MemberResidenceStreetAddress(self):
        return ["ResponseContactAddressLine_01",
                    "ResponseContactAddressLine_02"]

    def N4_MemberCity_State_ZIPCode(self):
        return ["AdditionalPatientInformationContactCityName_01",
                    "AdditionalPatientInformationContactStateCode_02",
                    "AdditionalPatientInformationContactPostalZoneorZIPCode_03",
                    "LocationQualifier_05",
                    "LocationIdentifier_06"]

    def DMG_MemberDemographics(self):
        return ["DateTimePeriodFormatQualifier_01",
                    "DependentBirthDate_02",
                    "DependentGenderCode_03"]

### Parsing the File

In [5]:
file_path = "MCE834Sample_2_copy.txt"
file2 = edi_parser(file_path)
parsed_data = file2.parse_834()

# new_dict = json.loads(parsed_data)

# print(new_dict["NM1"])

{
    "ISA": {
        "AuthorizationInformationQualifier_1": "00",
        "AuthorizationInformation_2": "          ",
        "SecurityInformationQualifier_3": "00",
        "SecurityInformation_4": "          ",
        "SenderIDQualifier_5": "ZZ",
        "InterchangeSenderID_6": "386028429      ",
        "ReceiverIDQualifier_7": "30",
        "InterchangeReceiverID_8": "382328142      ",
        "InterchangeDate_9": "050221",
        "InterchangeTime_10": "0602",
        "InterchangeControlStandardsIdentifier_11": "U",
        "InterchangeControlVersionNumber_12": "00501",
        "InterchangeControlNumber_13": "000012345",
        "AcknowledgementRequested_14": "0",
        "UsageIndicator_15": "P",
        "ComponentElementSeparator_16": ":"
    },
    "NM1": {
        "EntityIdentifierCode_01": "IL",
        "EntityTypeQualifier_02": "1",
        "ResponseContactLastorOrganizationName_03": "DOE",
        "ResponseContactFirstName_04": "JOHN",
        "ResponseContactMiddleName

## Generating the 834 text file

In [ ]:
class edi_generator:
    
    def __init__(self,jsonData):
        self.res_dict = json.loads(jsonData)
    
    def generate_834(self, w_file_path):
        
        segments = []
        for key, values in self.res_dict.items():
                segment_data = [key] + [*values.values()]
                segment = '*'.join(map(str, segment_data))
                segments.append(segment)

        #Join the segments with '~' delimiter
        data_834 = '~'.join(segments)
        #print(data)
        # Write the data to the file
        with open(w_file_path, "w") as file:
            file.write(data_834)
        
        return data_834

In [ ]:
w_file_path = "rev_Untitled.txt"
file_gen = edi_generator(parsed_data)
result_834 = file_gen.generate_834(w_file_path)

print(result_834)

## Data Model

In [ ]:
from typing import Any, TypeVar, Type, cast


T = TypeVar("T")


def from_str(x: Any) -> str:
    assert isinstance(x, str)
    return x


def to_class(c: Type[T], x: Any) -> dict:
    assert isinstance(x, c)
    return cast(Any, x).to_dict()


class NM1:
    entity_identifier_code_01: str
    entity_type_qualifier_02: str
    response_contact_lastor_organization_name_03: str
    response_contact_first_name_04: str
    response_contact_middle_name_05: str
    identification_code_qualifier_08: str
    response_contact_identifier_09: str

    def __init__(self, entity_identifier_code_01: str, entity_type_qualifier_02: str, response_contact_lastor_organization_name_03: str, response_contact_first_name_04: str, response_contact_middle_name_05: str, identification_code_qualifier_08: str, response_contact_identifier_09: str) -> None:
        self.entity_identifier_code_01 = entity_identifier_code_01
        self.entity_type_qualifier_02 = entity_type_qualifier_02
        self.response_contact_lastor_organization_name_03 = response_contact_lastor_organization_name_03
        self.response_contact_first_name_04 = response_contact_first_name_04
        self.response_contact_middle_name_05 = response_contact_middle_name_05
        self.identification_code_qualifier_08 = identification_code_qualifier_08
        self.response_contact_identifier_09 = response_contact_identifier_09

    def to_dict(self) -> dict:
        result: dict = {}
        result["EntityIdentifierCode_01"] = from_str(self.entity_identifier_code_01)
        result["EntityTypeQualifier_02"] = from_str(str(self.entity_type_qualifier_02))
        result["ResponseContactLastorOrganizationName_03"] = from_str(self.response_contact_lastor_organization_name_03)
        result["ResponseContactFirstName_04"] = from_str(self.response_contact_first_name_04)
        result["ResponseContactMiddleName_05"] = from_str(self.response_contact_middle_name_05)
        result["IdentificationCodeQualifier_08"] = from_str(str(self.identification_code_qualifier_08))
        result["ResponseContactIdentifier_09"] = from_str(str(self.response_contact_identifier_09))
        return result

In [ ]:
class ISA:
    authorization_information_qualifier_1: str
    authorization_information_2: str
    security_information_qualifier_3: str
    security_information_4: str
    sender_id_qualifier_5: str
    interchange_sender_id_6: str
    receiver_id_qualifier_7: str
    interchange_receiver_id_8: str
    interchange_date_9: str
    interchange_time_10: str
    interchange_control_standards_identifier_11: str
    interchange_control_version_number_12: str
    interchange_control_number_13: str
    acknowledgement_requested_14: int
    usage_indicator_15: str
    component_element_separator_16: str

    def __init__(self, authorization_information_qualifier_1: str, authorization_information_2: str, security_information_qualifier_3: str, security_information_4: str, sender_id_qualifier_5: str, interchange_sender_id_6: str, receiver_id_qualifier_7: str, interchange_receiver_id_8: str, interchange_date_9: str, interchange_time_10: str, interchange_control_standards_identifier_11: str, interchange_control_version_number_12: str, interchange_control_number_13: str, acknowledgement_requested_14: str, usage_indicator_15: str, component_element_separator_16: str) -> None:
        self.authorization_information_qualifier_1 = authorization_information_qualifier_1
        self.authorization_information_2 = authorization_information_2
        self.security_information_qualifier_3 = security_information_qualifier_3
        self.security_information_4 = security_information_4
        self.sender_id_qualifier_5 = sender_id_qualifier_5
        self.interchange_sender_id_6 = interchange_sender_id_6
        self.receiver_id_qualifier_7 = receiver_id_qualifier_7
        self.interchange_receiver_id_8 = interchange_receiver_id_8
        self.interchange_date_9 = interchange_date_9
        self.interchange_time_10 = interchange_time_10
        self.interchange_control_standards_identifier_11 = interchange_control_standards_identifier_11
        self.interchange_control_version_number_12 = interchange_control_version_number_12
        self.interchange_control_number_13 = interchange_control_number_13
        self.acknowledgement_requested_14 = acknowledgement_requested_14
        self.usage_indicator_15 = usage_indicator_15
        self.component_element_separator_16 = component_element_separator_16

    def to_dict(self) -> dict:
        result: dict = {}
        result["AuthorizationInformationQualifier_1"] = from_str(self.authorization_information_qualifier_1)
        result["AuthorizationInformation_2"] = from_str(self.authorization_information_2)
        result["SecurityInformationQualifier_3"] = from_str(self.security_information_qualifier_3)
        result["SecurityInformation_4"] = from_str(self.security_information_4)
        result["SenderIDQualifier_5"] = from_str(self.sender_id_qualifier_5)
        result["InterchangeSenderID_6"] = from_str(self.interchange_sender_id_6)
        result["ReceiverIDQualifier_7"] = from_str(str(self.receiver_id_qualifier_7))
        result["InterchangeReceiverID_8"] = from_str(self.interchange_receiver_id_8)
        result["InterchangeDate_9"] = from_str(self.interchange_date_9)
        result["InterchangeTime_10"] = from_str(self.interchange_time_10)
        result["InterchangeControlStandardsIdentifier_11"] = from_str(self.interchange_control_standards_identifier_11)
        result["InterchangeControlVersionNumber_12"] = from_str(self.interchange_control_version_number_12)
        result["InterchangeControlNumber_13"] = from_str(self.interchange_control_number_13)
        result["AcknowledgementRequested_14"] = from_str(str(self.acknowledgement_requested_14))
        result["UsageIndicator_15"] = from_str(self.usage_indicator_15)
        result["ComponentElementSeparator_16"] = from_str(self.component_element_separator_16)
        return result

In [ ]:
NM1_obj = NM1("IL", "1", "DOE", "JOHN", "P", "", "")
nm1_dict = NM1_obj.to_dict()
print(nm1_dict)

ISA_obj = ISA('00','','00','','ZZ','1234','34','54','45','54','545','545','545','545','234','243' )
isa_dict = ISA_obj.to_dict()
print(isa_dict)


In [ ]:
class CombinedObject(NM1, ISA):
    def __init__(self, isa_obj: ISA,  nm1_obj: NM1):
        self.isa_obj = isa_obj
        self.nm1_obj = nm1_obj
        self.res_dict = {}
        self.json_d = None

    def to_dict(self):
        self.res_dict =  {
            "ISA": self.isa_obj.to_dict(),
            "NM1": self.nm1_obj.to_dict(),
        }
        return self.res_dict
    
    def to_json(self):
        self.res_dict =  {
            "ISA": self.isa_obj.to_dict(),
            "NM1": self.nm1_obj.to_dict(),
        }
        self.json_d = json.dumps(self.res_dict , indent = 4)
        return self.json_d
        
    
    def generate_834(self, w_file_path):
        
        segments = []
        for key, values in self.res_dict.items():
                segment_data = [key] + [*values.values()]
                segment = '*'.join(map(str, segment_data))
                segments.append(segment)

        #Join the segments with '~' delimiter
        data_834 = '~'.join(segments)
        #print(data)
        # Write the data to the file
        with open(w_file_path, "w") as file:
            file.write(data_834)
        
        return data_834

In [ ]:
combined_object = CombinedObject(ISA_obj,NM1_obj)

# print(combined_object.to_dict())
print(combined_object.to_json())

fp = "gen_from_class.txt"


combined_object.generate_834(file_path)